In [1]:
import xarray as xr
import accessvis
import glob
import iris
from esmvalcore.preprocessor import regrid
from ncdata.iris_xarray import cubes_from_xarray
from scipy.ndimage import gaussian_filter

import seaborn as sns

import warnings
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import os
import xarray as xr
from tqdm import tqdm_notebook as tqdm
import dask.bag as dasb
from dask.diagnostics import ProgressBar
from scipy import signal
from scipy.stats import spearmanr, norm
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import xskillscore as xskill
import cmocean
import cartopy.feature as cfeature
import matplotlib.colors as mcolors

import sys
sys.path.append('/home/548/cxc548/lib/python/bom-climate-change-variability-and-extreme-toolbox')

In [ ]:
# Trigger Dask Client

In [ ]:
import dask.distributed as dask

# client = dask.Client(threads_per_worker=1,n_workers=4, memory_limit='64GB')
client = dask.Client(threads_per_worker=1)
client

# Method definition

In [2]:
def calc_trend(xarr, start,end,length):
    #loop over years
    for i in np.arange(start,end,1):
        startyear=i
        endyear=i+length
        #extract window
        arr = xarr.sel(year=slice(str(startyear),str(endyear)))
    
        timedim=np.arange(0,len(arr.year))
        arr['year'] = timedim
        #call polyfit to calculate trend over the window. multiply by 10 so we get units of trend per decade
        trnd = ((arr.polyfit(dim='year',deg=1,skipna=True)).polyfit_coefficients.isel(degree=0))*10
 
        #if it's the first year, copy the trend to trnarr
        if i == start:
            trnarr = trnd
        #if it's the subsequent years, concatenate the trend to trnarr
        else:
            trnarr = xr.concat([trnarr,trnd],dim='year')

    trnarr['year'] = np.arange(start,end,1)

    return trnarr

In [3]:
def regrid_dataset(data):
    cube_data=cubes_from_xarray(data)[0]
    cube_regrided = regrid(cube_data, target_grid="1x1", scheme="linear")
    return xr.DataArray.from_iris(cube_regrided)

In [4]:
def generate_rgba(data, lon, lat, vmax=0.2, vmin=-0.2, Alpha=0.65):
    
    lon2d, lat2d = np.meshgrid(lon, lat)
    
    cmap = cmocean.cm.balance
    
    norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
    
    
    fig, ax = plt.subplots(figsize=(20, 10), dpi=100)
    cf = ax.contourf(lon2d, lat2d, data, cmap=cmap, norm=norm, levels=30)
    ax.axis('off')
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

    fig.canvas.draw()
    rgba = np.frombuffer(fig.canvas.tostring_argb(), dtype=np.uint8)
    rgba = rgba.reshape(fig.canvas.get_width_height()[::-1] + (4,))  # (H, W, 4)

    rgba = rgba[:, :, [1, 2, 3, 0]]

    # mask land
    img = rgba.astype(float) / 255.0
    tol = 0.99
    mask_white = (img[..., :3] > tol).all(axis=-1)
    rgba[mask_white, 3] = rgba[mask_white, 3] * 0.001  

    rgba[:,:,3] = rgba[:,:,3]*Alpha
    plt.close(fig)
    
    return rgba

# Load Data

In [17]:
model_pattern="/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r1i1p1f1/Omon/tos/gn/latest/*"
obs_pattern="/g/data/ct11/access-nri/replicas/esmvaltool/obsdata-v2/Tier2/NOAA-ERSSTv5/*"

ds_model=xr.open_mfdataset(model_pattern)
ds_obs=xr.open_mfdataset(obs_pattern)

# Data pre-processes

In [18]:
# Regrid data

regrided_model=regrid_dataset(ds_model)
regrided_obs=regrid_dataset(ds_obs)

/g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/iris/fileformats/cf.py:880: IrisCfMissingVarWarning: Missing CF-netCDF measure variable 'areacello', referenced by netCDF variable 'tos'
  warnings.warn(
/g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/iris/fileformats/cf.py:1475: IrisCfNonSpanningVarWarning: Ignoring variable lat_bnds referenced by variable lat: Dimensions ('time', 'lat', 'bnds') do not span ('lat',)
  warnings.warn(
/g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/iris/fileformats/cf.py:1475: IrisCfNonSpanningVarWarning: Ignoring variable lon_bnds referenced by variable lon: Dimensions ('time', 'lon', 'bnds') do not span ('lon',)
  warnings.warn(


In [19]:
# Calculate annual_mean

ds_model_ann=regrided_model.groupby('time.year').mean('time')
ds_obs_ann=regrided_obs.groupby('time.year').mean('time')

In [20]:
# Calculate 31 years trends

ds_model_ann_trend_30 = calc_trend(ds_model_ann,1850,1980,31)
ds_obs_ann_trend_30 = calc_trend(ds_obs_ann,1850,1980,31)

In [22]:
# Calculate Rolling(annually) SSTA
ds_obs = ds_obs.unify_chunks()
ds_obs_anoms = ds_obs.groupby('time.month') - ds_obs.sel(time=slice('1850','1900')).groupby('time.month').mean('time')
ds_obs_anoms_rolling = ds_obs_anoms.rolling(time=12, center=True).mean()

# ds_model = ds_model.unify_chunks()
# ds_model_anoms = ds_model.groupby('time.month') - ds_model.sel(time=slice('1850','1900')).groupby('time.month').mean('time')
# ds_model_anoms_rolling = ds_model_anoms.rolling(time=12, center=True).mean()

In [23]:
# To make video looks more smooth, we do some interpolation in trands data

year = ds_obs_ann_trend_30["year"].values
new_year = np.linspace(year.min(), year.max(), len(year) * 4)
ds_load=ds_obs_ann_trend_30.load()
ds_interp_obs = (
    ds_load
    .chunk({'year': -1})       
    .interp(year=new_year)    
    .chunk({'year': 10, 'lat': 180, 'lon': 360})  
)

year = ds_model_ann_trend_30["year"].values
new_year = np.linspace(year.min(), year.max(), len(year) * 4)
ds_load_model=ds_model_ann_trend_30.load()
ds_interp_model = (
    ds_load_model
    .chunk({'year': -1})             
    .interp(year=new_year)          
    .chunk({'year': 10, 'lat': 180, 'lon': 360})  
)



KeyboardInterrupt



In [ ]:
# Define some specific camera spots which going to use in the video

cam_aust={
    # 'translate': [0.0, 0.0, -22.071718],
    'translate': [0.0, 0.0, -14.071718],
    'rotate': [0.003478, -0.91839, 0.218037, 0.330141],
    'xyzrotate': [-149.899796, -37.438725, 170.021637],
    'fov': 45.0,
    'focus': [-0.014273, -0.037801, 0.001495]
}

cam_westpacific={
 'translate': [0.0, 0.0, -26.93169],
 'rotate': [0.00821, -0.998777, 0.000118, 0.048616],
 'xyzrotate': [179.96759, -5.573063, -179.056473],
 'fov': 45.0,
 'focus': [-0.014273, -0.037801, 0.001495]
}

cam_eastpacific={
 'translate': [0.0, 0.0, -26.93169],
 'rotate': [0.001449, -0.855247, 0.001781, -0.518191],
 'xyzrotate': [-179.437088, 62.419106, -179.464828],
 'fov': 45.0,
 'focus': [-0.014273, -0.037801, 0.001495]
}

# Generate Video

In [ ]:
path="/g/data/tm70/yz9299/ARE/video"
lv = accessvis.plot_earth(texture='bluemarble', background="white", vertical_exaggeration=20)
data_rolling=ds_anoms_rolling.isel(time=slice(6, -6))
data_trend=ds_obs_ann_trend_30
data_interp=ds_interp_obs


with lv.video(filename=os.path.join(path,'SSTA_test_part1_1.mp4'), quality=2, resolution=(600,600), width=600, height=600, fps=30) as v:

    # define parameters for video generation
    Y=0
    fps=v.
    FIRST_PART=3
    SECOND_PART=45
    THIRD_PART=20
    ALPHA=0.65
    ALPHA_TIME=1.5

    # define time for each part of the video
    time_list=[]
    
    time_fristpart=fps*FIRST_PART
    time_list.append(time_fristpart)
    
    time_secondpart=fps*SECOND_PART
    time_list.append(time_secondpart)

    time_thirdpart=fps*THIRD_PART
    time_list.append(time_thirdpart)

    time_total=sum(time_list)

    # Part one: start with spinning globe and annual SSTAs from 1850-present
    temp_alpha=0
    data=ds_anoms_rolling.isel(time=slice(6, -6))
    for Y in tqdm(range(0,-(time_fristpart+time_secondpart),-1)):
        lv.reset()
        if abs(Y)<time_fristpart:
            lv.rotate('y', Y)
            lv.render()
            # Y-=1
        elif abs(Y)>time_fristpart and abs(Y)<time_fristpart+time_secondpart:
            time_index=abs(Y)-time_fristpart
            # for time in tqdm(data.time.values[6:300]): 
            # if temp_alpha<ALPHA:
            #     temp_alpha+=ALPHA/(ALPHA_TIME*fps)
            if time_fristpart+time_secondpart-abs(Y) < ALPHA_TIME*fps and temp_alpha>0:
                # print(Y)
                temp_alpha-=ALPHA/(ALPHA_TIME*fps)
                # print(temp_alpha)
            elif abs(Y)-time_fristpart<ALPHA_TIME*fps and temp_alpha<ALPHA:
                temp_alpha+=ALPHA/(ALPHA_TIME*fps)
                
            rgba_obs_anoms=generate_rgba(data.tos.isel(time=time_index),data['lon'], data['lat'], vmax=1, vmin=-1, Alpha=temp_alpha)
            smoothed = gaussian_filter(rgba_obs_anoms, sigma=0.5)
            accessvis.update_earth_values(lv, dataMode=0, data=smoothed)
            lv.rotate('y', Y)
            lv.title("NOAA-ERSSTv5 SSTA")
            lv.render()

    lv.title("")
    # Zoom in to Australia
    lv.flyto(cam_aust,steps=150)

    # SSTA in Australian region
    for Y in tqdm(range(0,-time_thirdpart,-1)):
        time_index=abs(Y)
        
        if time_thirdpart-abs(Y) < ALPHA_TIME*fps and temp_alpha>0:
            # print(Y)
            temp_alpha-=ALPHA/(ALPHA_TIME*fps)
            # print(temp_alpha)
        elif abs(Y)-time_thirdpart<ALPHA_TIME*fps and temp_alpha<ALPHA:
            temp_alpha+=ALPHA/(ALPHA_TIME*fps)

        lon = data['lon']
        lat = data['lat']
        rgba_obs_anoms=generate_rgba(data.tos.isel(time=time_index), lon, lat, vmax=1, vmin=-1, Alpha=temp_alpha)
        smoothed = gaussian_filter(rgba_obs_anoms, sigma=0.5)
        accessvis.update_earth_values(lv, dataMode=0, data=smoothed)
        lv.render()

    # Zoom out to the western Pacific tropical region.
    lv.flyto(cam_westpacific,steps=100)

    # Pause at the end for 2 seconds
    for i in range(2*fps):
        lv.render()

    # show 31 years trends from 1850 to 1980, rotate gradually to the east
    temp_alpha=0
    test_data_range=len(data_interp.year)
    pos0=lv.camera()
    pos1=cam_eastpacific
    lv.title("Pacific SST Trend 31 years")
    for year in tqdm(range(test_data_range)):
        if year < ALPHA_TIME * fps and temp_alpha<ALPHA:
            temp_alpha+=ALPHA/(ALPHA_TIME*fps)
        # elif year>test_data_range-(ALPHA_TIME * fps) and temp_alpha>0:
        #     temp_alpha-=ALPHA/(ALPHA_TIME*fps)
        data = data_interp.isel(year=year)
        rgba=generate_rgba(data, data['lon'], data['lat'], Alpha=temp_alpha)
        smoothed = gaussian_filter(rgba, sigma=0.5)
        accessvis.update_earth_values(lv, dataMode=0, data=smoothed)
        L = year / (test_data_range - 1)
        lv.camlerp(pos0, pos1, L)
        
        lv.render()

    # Pause for 5 seconds
    for i in range(5*fps):
        lv.render()

# Generate Trend Video for different models

In [25]:
model_list=[ds_interp_obs,ds_interp_mdoel]

NameError: name 'ds_obs_interp' is not defined

In [ ]:
warnings.filterwarnings(
    "ignore",
    message="Sending large graph of size",
    category=UserWarning,
    module="distributed.client"
)

path="Path to store the video"# Path to store the video

for i, data in enumerate(model_list)
    lv = accessvis.plot_earth(texture='bluemarble', background="white", vertical_exaggeration=20)
    data_interp=data
    warnings.filterwarnings(
        "ignore",
        message="Sending large graph of size",
        category=UserWarning,
        module="distributed.client"
    )
    
    with lv.video(filename=os.path.join(path,f"Trend_{i}.mp4"), quality=2, resolution=(600,600), width=600, height=600, fps=30) as v:
        ALPHA=0.65
        ALPHA_TIME=1.5
        
        fps=v.framerate
        lv.camera(cam_eastpacific)
        lv.title("NOAA-ERSSTv5")
    
        temp_alpha=ALPHA
        data=data_interp.isel(year=-1)
        rgba_east=generate_rgba(data, data['lon'], data['lat'], Alpha=temp_alpha)
        smoothed = gaussian_filter(rgba_east, sigma=0.5)
        accessvis.update_earth_values(lv, dataMode=0, data=smoothed)
        for i in tqdm(range(6*fps)):
            if 6*fps-i<ALPHA_TIME*fps:
                temp_alpha-=ALPHA/(ALPHA_TIME*fps)
                rgba_east=generate_rgba(data, data['lon'], data['lat'], Alpha=temp_alpha)
                smoothed = gaussian_filter(rgba_east, sigma=0.5)
                accessvis.update_earth_values(lv, dataMode=0, data=smoothed)
            
            lv.render()
    
        lv.flyto(cam_westpacific,steps=3*fps)
    
        temp_alpha=0
        test_data_range=len(data_interp.year)
        pos0=lv.camera()
        pos1=cam_eastpacific
        for year in tqdm(range(test_data_range)):
            if year < ALPHA_TIME * fps and temp_alpha<ALPHA:
                temp_alpha+=ALPHA/(ALPHA_TIME*fps)
            data = data_interp.isel(year=year)
            rgba=generate_rgba(data, data['lon'], data['lat'], Alpha=temp_alpha)
            smoothed = gaussian_filter(rgba, sigma=0.5)
            accessvis.update_earth_values(lv, dataMode=0, data=smoothed)
            L = year / (test_data_range - 1)
            lv.camlerp(pos0, pos1, L)
    
            lv.render()
    
        for i in tqdm(range(3*fps)):
            lv.render()

# Concatenate videos

In [ ]:
# from moviepy.editor import VideoFileClip, clips_array
import cv2

video1 = os.path.join(path, "Trend_1.mp4")
video2 = os.path.join(path, "Trend_2.mp4")

# Open videos
cap1 = cv2.VideoCapture(video1)
cap2 = cv2.VideoCapture(video2)

# Get video parameters (using first video as reference)
fps = int(cap1.get(cv2.CAP_PROP_FPS))
width = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output video settings: width × 2 for horizontal concatenation
out = cv2.VideoWriter(
    os.path.join(path, "merged_side_by_side.mp4"),
    cv2.VideoWriter_fourcc(*'avc1'),
    fps,
    (width * 2, height)
)

# Get frame count from the shorter video
frame_count = int(min(cap1.get(cv2.CAP_PROP_FRAME_COUNT),
                      cap2.get(cv2.CAP_PROP_FRAME_COUNT)))

for i in range(frame_count):
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()
    
    if not (ret1 and ret2):
        break
    
    # Resize to match dimensions (prevent resolution mismatch)
    frame2 = cv2.resize(frame2, (width, height))
    
    # Horizontal concatenation
    combined = np.hstack((frame1, frame2))
    out.write(combined)

# Release resources
cap1.release()
cap2.release()
out.release()RetryClaude can make mistakes. Please double-check responses.